In [65]:
# !pip install torchsummary

In [66]:
##code
"""
Target : increase accuracy by adding BatchNorm
Result : 3,342 Params, test: 98.89%%, train 98.61% at 15th rpoch
Analysis: Performance improved from 94.74% to 98.61% , training speed improved
"""

'\nTarget : increase accuracy at 15th epoch, keeping same parameters\nResult : test: (94.74%)%, train 93.91 at 15th rpoch,3,258\nAnalysis: training speed improved, still very huge parameter\n'

In [67]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [68]:
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.convblock1 = nn.Sequential(
          nn.Conv2d(in_channels=1, out_channels=2 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(2),
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False),
          nn.BatchNorm2d(8),
          nn.ReLU(),
        )
        self.pool1 = nn.Sequential( 
          nn.MaxPool2d(2,2),
        )
        self.convblock2 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=2 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(2),
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(8),
          nn.ReLU(),        
        )
        self.convblock3 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=2 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(2),
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), 
          nn.BatchNorm2d(4),
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), 
          nn.BatchNorm2d(8),
          nn.ReLU(),
        )
        self.pool2 = nn.Sequential(
          nn.MaxPool2d(2,2),
        )
        self.convblock4 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=8 ,kernel_size=3,padding=1, bias=False), 
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=8 ,kernel_size=3,padding=1, bias=False), 
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=10 ,kernel_size=3,padding=0, bias=False), 
          # nn.ReLU()
        )
        # self.gap = nn.AvgPool2d(10)

    def forward(self, x):
        x = self.convblock1(x)
        # print (x.shape)
        x = self.pool1(x)
        # print (x.shape)
        x= self.convblock2(x)
        # print (x.shape)
        x= self.convblock3(x)
        # print (x.shape)
        x= self.pool2(x)
        # print (x.shape)
        x= self.convblock4(x)
        # print (x.shape)
        x = x.view(-1,10)
        x = F.log_softmax(x)
        return x 


In [69]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 26, 26]              18
       BatchNorm2d-2            [-1, 2, 26, 26]               4
              ReLU-3            [-1, 2, 26, 26]               0
            Conv2d-4            [-1, 4, 24, 24]              72
       BatchNorm2d-5            [-1, 4, 24, 24]               8
              ReLU-6            [-1, 4, 24, 24]               0
            Conv2d-7            [-1, 8, 22, 22]             288
       BatchNorm2d-8            [-1, 8, 22, 22]              16
              ReLU-9            [-1, 8, 22, 22]               0
        MaxPool2d-10            [-1, 8, 11, 11]               0
           Conv2d-11            [-1, 2, 11, 11]             144
      BatchNorm2d-12            [-1, 2, 11, 11]               4
             ReLU-13            [-1, 2, 11, 11]               0
           Conv2d-14            [-1, 4,

/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_51286/2979057407.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [70]:
transforms_ = transforms.Compose([
                        transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

In [71]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 7, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
                        # transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [72]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1)
        correct+=pred.eq(target).sum().item()
    acc = 100*correct/len(train_loader.dataset)
    print(f'Train Accuracy: {acc:.2f}%')
        
def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _,(data, target) in enumerate(test_loader):
            data, target = data.float().to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [73]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(1, 15):
    print(f'Epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)


Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_51286/2979057407.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
loss=0.06319721043109894 batch_id=937: 100%|██████████| 938/938 [00:24<00:00, 38.36it/s] 


Train Accuracy: 80.74%
Test set: Average loss: 0.1643, Accuracy: 9469/10000 (94.69%)
Epoch: 2


loss=0.3645118772983551 batch_id=937: 100%|██████████| 938/938 [00:25<00:00, 36.34it/s]  


Train Accuracy: 95.98%
Test set: Average loss: 0.0873, Accuracy: 9725/10000 (97.25%)
Epoch: 3


...

Train Accuracy: 98.50%
Test set: Average loss: 0.0446, Accuracy: 9861/10000 (98.61%)
Epoch: 13


loss=0.001545490580610931 batch_id=937: 100%|██████████| 938/938 [00:37<00:00, 25.21it/s] 


Train Accuracy: 98.40%
Test set: Average loss: 0.0451, Accuracy: 9853/10000 (98.53%)
Epoch: 14


loss=0.0062868548557162285 batch_id=937: 100%|██████████| 938/938 [00:39<00:00, 23.82it/s]


Train Accuracy: 98.61%
Test set: Average loss: 0.0389, Accuracy: 9889/10000 (98.89%)
